In [1]:
import glob
from tqdm import tqdm
from tqdm import trange
import matplotlib.pyplot as plt
import matplotlib as mpl
import torch
import numpy as np
import json


In [2]:
import sys
sys.path.append('../')
sys.path.append('../utils')
from models import imagenet_resnet
import load
import optimize

In [4]:
# this loads the state dictionary from all the checkpoints in the folder, averages the last n, tensor by tensor,
# and returns the average state dictionary

def load_cache_ave_n(save_dir, experiment, expid, metric, n):
    exp_path = f"{save_dir}/{experiment}/{expid}"
    step_names = glob.glob(f"{exp_path}/ckpt/*.tar")
    step_list = [int(s.split(".tar")[0].split("step")[1]) for s in step_names]
    steps, cache = [], []
    i = 0
    current_check = 0
    temp = sorted(list(zip(step_names, step_list)), key=lambda x: x[1])
    for in_filename, step in tqdm(temp[-n:]):
        
        checkpoint = torch.load(in_filename)
        
        if i == 0:
            print("There are ",  len(checkpoint["model_state_dict"].keys()), " tensors in this model" )
            i+=1
            
        if current_check == 0:
            new_model = checkpoint["model_state_dict"].copy()
        else:
            for key in checkpoint["model_state_dict"].keys():
                new_model[key] = new_model[key]+checkpoint["model_state_dict"][key]
        current_check+=1
        
    for key in new_model.keys():
        new_model[key]=new_model[key]/current_check
    
    print("Averaged over ",current_check , "checkpoints")
    return new_model


In [ ]:
# This gives the accuracy as a function of how many checkpoints this has been averaged over, from the last one
dir_exp="../results" ##Here the directory where the experiments are stored
metric = "test_accuracy1"
input_shape, num_classes = load.dimension("tiny-imagenet")
CACHE_PATH = dir_exp
all_accs = []
experiment=  "run-ECD"  ##name of the saved experiment


expid = str(1)    

accuracies = []
for n_to_average in range(1,11):
    # This returns the averaged parameters
    averaged_model = load_cache_ave_n(CACHE_PATH, experiment, expid, metric, n_to_average)


    model = load.model('resnet18','imagenet')(
            input_shape=input_shape, num_classes=num_classes, pretrained=False,
            model_dir="",
        )
    model.load_state_dict(averaged_model)
    torch.cuda.set_device(0)
    model.to(torch.device("cuda"))

    #Now evaluate the performance of this averaged network
    ## Number epoch does not mean anything here 

    test_loader = load.dataloader(
        dataset="imagenet",
        batch_size=128,
        train=False,
        workers=2,
        datadir="../data",#directory where the dataset is stored
        tpu=False,
    )
    res = optimize.eval(model, load.loss("ce"), test_loader, torch.device("cuda"), False, 0)
    accuracies.append(res[1])
all_accs.append(accuracies)
    
with open(experiment+"accuracies.json", "w") as outfile:
    json.dump(all_accs, outfile)


In [ ]:
experiment = "run-ECD"
with open(experiment+"accuracies.json", "r") as outfile:
    all_accs = json.load(outfile)

maxes  =[]
for accuracies in all_accs:
    maxes.append(max(accuracies))
print(max(maxes))